## Sentitment by Finetuning Bert model

### Data Preprocessing

In [1]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# Download data
import requests
request = requests.get("https://drive.google.com/uc?export=download&id=1wHt8PsMLsfX5yNSqrt2fSTcb8LEiclcf")
with open("data.zip", "wb") as file:
    file.write(request.content)

# Unzip data
import zipfile
with zipfile.ZipFile('data.zip') as zip:
    zip.extractall('data')

In [34]:
 # Load data and set labels
data_complaint = pd.read_csv('data/complaint1700.csv')
data_complaint['label'] = 0
data_non_complaint = pd.read_csv('data/noncomplaint1700.csv')
data_non_complaint['label'] = 1

# Concatenate complaining and non-complaining data
data = pd.concat([data_complaint, data_non_complaint], axis=0).reset_index(drop=True)

# Drop 'airline' column
data.drop(['airline'], inplace=True, axis=1)

# Display 5 random samples
print(list(data.head(5)['tweet']))
print(list(data.head(5)['label']))
data.sample(5)

["@united I'm having issues. Yesterday I rebooked for 24 hours after I was supposed to fly, now I can't log on &amp; check in. Can you help?", "@united kinda feel like the $6.99 you charge for in flight Wi-Fi is ridiculous. AND it sucks, slow, or doesn't work. #anythingtomakeabuck", 'Livid in Vegas, delayed, again&amp; again&amp;again, @SouthwestAir decided to cancel a flight and combine two, then waited on crew, now pilots.', '@united the most annoying man on earth is on my flight. what can you do to help me?', "@united The last 2 weeks I've flown wit u, you have given me 4 reasons to convince me it was a bad decision. Time 2 go back 2 @SouthwestAir"]
[0, 0, 0, 0, 0]


,id,tweet,label
537,134256,@AmericanAir I just found your breach! I don't...,0
1822,14116,@flyLAXairport @AmericanAir @FlyEIA Odd that t...,1
2538,78337,Hey @SouthwestAir will you have more nonstop f...,1
395,157276,Dad: Delta Charged Me $88 Fee to Sit Next to 4...,0
2156,38242,Only @JetBlue would value investors over custo...,1


In [18]:
data.count()

id       3400
tweet    3400
label    3400
dtype: int64

In [14]:
max_tweet_length = max([len(each) for each in list(data['tweet'])])
max_tweet_length

187

In [4]:
from sklearn.model_selection import train_test_split

X = data.tweet.values
y = data.label.values

X_train, X_val, y_train, y_val =\
    train_test_split(X, y, test_size=0.1, random_state=2022)

In [15]:
# Load test data
test_data = pd.read_csv('data/test_data.csv')

# Keep important columns
test_data = test_data[['id', 'tweet']]

# Display 5 samples from the test data
test_data.sample(5)

,id,tweet
1091,42892,@AmericanAir please tell me how it's possible ...
3473,131396,Hiiiii @DeltaAssist I've been on hold for over...
2715,103943,@BTSullivan91 @AmericanAir why are they stuck ...
780,31230,@AmericanAir Why the heck did you yank your Wi...
1065,42436,Sad but very true. @united http://t.co/O282zo...


In [114]:
max_tweet_length = max([len(each.split(' ')) for each in list(test_data['tweet'])])
max_tweet_length

32

### Bert model finetuning

In [115]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

pretrained_bert_model_name = 'bert-base-uncased'
pretrained_bert_tokenizer = AutoTokenizer.from_pretrained(pretrained_bert_model_name)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /Users/sean.huang/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncase

In [116]:
from datasets import Dataset

def prepare_classification_labels(samples):
    inputs = pretrained_bert_tokenizer(samples['tweet'], truncation=True, padding='max_length', max_length=32, is_split_into_words=False)
    
    # print('inputs', inputs)
    # print('inputs', inputs.keys())
    
    return inputs

train_datasets = Dataset.from_pandas(data[:300]).map(prepare_classification_labels, batched=True, batch_size=100).remove_columns(['id', 'tweet'])
validation_datasets = Dataset.from_pandas(data[300:340]).map(prepare_classification_labels, batched=True, batch_size=10).remove_columns(['id', 'tweet'])

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [81]:
pretrained_bert_model = AutoModelForSequenceClassification.from_pretrained(pretrained_bert_model_name, num_labels=2, id2label={0:'negative', 1:'positive'}, label2id={'negative':0, 'positive':1})

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /Users/sean.huang/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "negative",
    "1": "positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 0,
    "positive": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  

In [82]:
from transformers import TrainingArguments, Trainer
import torch 

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')   # mps is for Apple M1 GPU
device

device(type='mps')

In [83]:
from transformers import EarlyStoppingCallback

args = TrainingArguments(
    output_dir='./sentiment_test_model',
    per_device_train_batch_size=32,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    logging_strategy='steps',
    evaluation_strategy='steps',
    eval_steps = 10,
    load_best_model_at_end=True,
    report_to='none'
)

    
trainer = Trainer(
    model=pretrained_bert_model,
    args=args,
    train_dataset=train_datasets,
    eval_dataset=validation_datasets,
    tokenizer=pretrained_bert_tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()
trainer.save_model()

PyTorch: setting up devices
torch.distributed process group is initialized, but local_rank == -1. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch
***** Running training *****
  Num examples = 300
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 100


Step,Training Loss,Validation Loss
10,No log,0.025234
20,0.252200,0.003022
30,0.012900,0.001119
40,0.002500,0.000690
50,0.001100,0.000518
60,0.000800,0.000436
70,0.000600,0.000385
80,0.000500,0.000358
90,0.000500,0.000343
100,0.000400,0.000338


***** Running Evaluation *****
  Num examples = 40
  Batch size = 4
***** Running Evaluation *****
  Num examples = 40
  Batch size = 4
***** Running Evaluation *****
  Num examples = 40
  Batch size = 4
***** Running Evaluation *****
  Num examples = 40
  Batch size = 4
***** Running Evaluation *****
  Num examples = 40
  Batch size = 4
***** Running Evaluation *****
  Num examples = 40
  Batch size = 4
***** Running Evaluation *****
  Num examples = 40
  Batch size = 4
***** Running Evaluation *****
  Num examples = 40
  Batch size = 4
***** Running Evaluation *****
  Num examples = 40
  Batch size = 4
***** Running Evaluation *****
  Num examples = 40
  Batch size = 4


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ./sentiment_test_model
Configuration saved in ./sentiment_test_model/config.json
Model weights saved in ./sentiment_test_model/pytorch_model.bin
tokenizer config file saved in ./sentiment_test_model/tokenize

## Restaurant ASC(Aspect based Sentiment Classification)

### Preprocess training data

In [1]:
import json
import pandas as pd

rest_asc_training_file = '/Users/sean.huang/Documents/my/books and courses/ztgg/Shared Code/BERT-for-ABSA/asc/rest/train.json'
rest_asc_training_samples = json.load(open(rest_asc_training_file))
len(rest_asc_training_samples)

3452

In [2]:
rest_asc_training_samples['1592_0']

{'polarity': 'positive',
 'term': 'server',
 'id': '1592_0',
 'sentence': 'Our server was very helpful and friendly.'}

In [3]:
dict_rest_asc_training_samples = {'sample_id':[], 'sentence':[], 'term':[], 'label':[]}

for key in rest_asc_training_samples.keys():
    dict_rest_asc_training_samples['sample_id'].append(key)
    dict_rest_asc_training_samples['sentence'].append(rest_asc_training_samples[key]['sentence'])
    dict_rest_asc_training_samples['term'].append(rest_asc_training_samples[key]['term'])
    dict_rest_asc_training_samples['label'].append(0 if rest_asc_training_samples[key]['polarity'] == 'negative' else 1 if rest_asc_training_samples[key]['polarity'] == 'positive' else 2)
    
# dict_rest_asc_training_samples

In [4]:
df_rest_asc_training_samples = pd.DataFrame.from_dict(dict_rest_asc_training_samples)
df_rest_asc_training_samples

,sample_id,sentence,term,label
0,1592_0,Our server was very helpful and friendly.,server,1
1,1940_1,"Super friendly and knowledgable staff, fabulou...",staff,1
2,1940_0,"Super friendly and knowledgable staff, fabulou...",jazz brunch,1
3,1940_3,"Super friendly and knowledgable staff, fabulou...",live jazz,1
4,1940_2,"Super friendly and knowledgable staff, fabulou...",bistro fare,1
...,...,...,...,...
3447,3139_2,Where tanks in other Chinatown restaurants dis...,tanks,0
3448,3139_1,Where tanks in other Chinatown restaurants dis...,dim sum,2
3449,3139_0,Where tanks in other Chinatown restaurants dis...,tanks,1
3450,2818_0,This was my frist time at Cafe St. Bart's and ...,service,1


In [5]:
max_sent_length = max([len(each.split(' ')) for each in list(df_rest_asc_training_samples['sentence'])])
max_sent_length

69

In [6]:
from collections import Counter
Counter(df_rest_asc_training_samples['label'])

Counter({1: 2094, 0: 779, 2: 579})

### Load and finetune pretrained Bert model

In [8]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

pretrained_bert_model_name = 'bert-base-uncased'
pretrained_bert_tokenizer = AutoTokenizer.from_pretrained(pretrained_bert_model_name)

In [9]:
tokens = pretrained_bert_tokenizer(list(df_rest_asc_training_samples['sentence'])[0], is_split_into_words=False)
tokens

{'input_ids': [101, 2256, 8241, 2001, 2200, 14044, 1998, 5379, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [10]:
sentence1 = list(df_rest_asc_training_samples['sentence'])[0]
term1 = list(df_rest_asc_training_samples['term'])[0]
sentence1, term1

('Our server was very helpful and friendly.', 'server')

In [11]:
tokens = pretrained_bert_tokenizer(sentence1, term1, is_split_into_words=False)
tokens

{'input_ids': [101, 2256, 8241, 2001, 2200, 14044, 1998, 5379, 1012, 102, 8241, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [12]:
tokens = pretrained_bert_tokenizer(sentence1, term1, truncation=True, padding='max_length', max_length=16, is_split_into_words=False)
tokens

{'input_ids': [101, 2256, 8241, 2001, 2200, 14044, 1998, 5379, 1012, 102, 8241, 102, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]}

In [13]:
from datasets import Dataset

def prepare_asc_classification_labels(samples):
    inputs = pretrained_bert_tokenizer(samples['sentence'], samples['term'], truncation=True, padding='max_length', max_length=128, is_split_into_words=False)
    
    # print('inputs', inputs)
    # print('inputs', inputs.keys())
    
    return inputs

asc_train_datasets = Dataset.from_pandas(df_rest_asc_training_samples[:3000]).map(prepare_asc_classification_labels, batched=True, batch_size=1000).remove_columns(['sample_id', 'sentence', 'term'])
asc_validation_datasets = Dataset.from_pandas(df_rest_asc_training_samples[3000:]).map(prepare_asc_classification_labels, batched=True, batch_size=100).remove_columns(['sample_id', 'sentence', 'term'])

Parameter 'function'=<function prepare_asc_classification_labels at 0x15b37d550> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [14]:
asc_train_datasets

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3000
})

In [15]:
asc_validation_datasets

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 452
})

In [16]:
import torch 

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')   # mps is for Apple M1 GPU
device

device(type='mps')

In [17]:
from transformers import TrainingArguments, Trainer, DataCollatorForTokenClassification
import torch

pretrained_bert_model = AutoModelForSequenceClassification.from_pretrained(pretrained_bert_model_name, num_labels=3, id2label={0:'negative', 1:'positive', 2:'neutral'}, label2id={'negative':0, 'positive':1, 'neutral':2})

pretrained_bert_model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [19]:
import numpy as np
from datasets import load_metric
metric = load_metric('accuracy')
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [20]:
from transformers import EarlyStoppingCallback

args = TrainingArguments(
    output_dir='./rest_asc_model',
    per_device_train_batch_size=128,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    logging_strategy='steps',
    logging_steps=25,
    evaluation_strategy='steps',
    eval_steps = 25,
    load_best_model_at_end=True,
    metric_for_best_model='eval_accuracy',
    report_to='none'
)
    
trainer = Trainer(
    model=pretrained_bert_model,
    args=args,
    train_dataset=asc_train_datasets,
    eval_dataset=asc_validation_datasets,
    tokenizer=pretrained_bert_tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()
trainer.save_model()

torch.distributed process group is initialized, but local_rank == -1. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch
/Users/sean.huang/miniconda3/envs/m1_new/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3000
  Num Epochs = 20
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 480


Step,Training Loss,Validation Loss,Accuracy
25,0.848100,0.717173,0.701327
50,0.569900,0.734159,0.712389
75,0.384300,0.646147,0.774336
100,0.261400,0.634331,0.780973
125,0.176700,0.640574,0.800885
150,0.109600,0.831753,0.785398
175,0.087300,0.975085,0.783186
200,0.058700,0.791748,0.798673
225,0.038600,0.983296,0.809735
250,0.027200,0.944392,0.809735


***** Running Evaluation *****
  Num examples = 452
  Batch size = 16
***** Running Evaluation *****
  Num examples = 452
  Batch size = 16
***** Running Evaluation *****
  Num examples = 452
  Batch size = 16
***** Running Evaluation *****
  Num examples = 452
  Batch size = 16
***** Running Evaluation *****
  Num examples = 452
  Batch size = 16
***** Running Evaluation *****
  Num examples = 452
  Batch size = 16
***** Running Evaluation *****
  Num examples = 452
  Batch size = 16
***** Running Evaluation *****
  Num examples = 452
  Batch size = 16
***** Running Evaluation *****
  Num examples = 452
  Batch size = 16
***** Running Evaluation *****
  Num examples = 452
  Batch size = 16
***** Running Evaluation *****
  Num examples = 452
  Batch size = 16
***** Running Evaluation *****
  Num examples = 452
  Batch size = 16
***** Running Evaluation *****
  Num examples = 452
  Batch size = 16
***** Running Evaluation *****
  Num examples = 452
  Batch size = 16
***** Running Evalua

### Load finetune Bert model for Rest NER task, and do prediction

In [22]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

finetuned_rest_asc_tokenizer = AutoTokenizer.from_pretrained('./rest_asc_model')
finetuned_rest_asc_model = AutoModelForSequenceClassification.from_pretrained('./rest_asc_model', num_labels=3)

Didn't find file ./rest_asc_model/added_tokens.json. We won't load it.
loading file ./rest_asc_model/vocab.txt
loading file ./rest_asc_model/tokenizer.json
loading file None
loading file ./rest_asc_model/special_tokens_map.json
loading file ./rest_asc_model/tokenizer_config.json
loading configuration file ./rest_asc_model/config.json
Model config BertConfig {
  "_name_or_path": "./rest_asc_model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "negative",
    "1": "positive",
    "2": "neutral"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 0,
    "neutral": 2,
    "positive": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,


In [33]:
sentiment_labels = finetuned_rest_asc_model.config.id2label
sentiment_labels

{0: 'negative', 1: 'positive', 2: 'neutral'}

In [49]:
import numpy as np
tokens = finetuned_rest_asc_tokenizer('very terrible service, price is high, but taste is good.', is_split_into_words=False)

output = finetuned_rest_asc_model(torch.tensor([tokens['input_ids']]))
print('Logits:', output.logits)
print('Sentiment: ', sentiment_labels[np.argmax(output.logits.tolist())])

Logits: tensor([[ 4.3336, -0.8536, -3.9473]], grad_fn=<AddmmBackward0>)
Sentiment:  negative


In [50]:
tokens = finetuned_rest_asc_tokenizer('very terrible service, price is high, but taste is good.', 'service', is_split_into_words=False)

output = finetuned_rest_asc_model(torch.tensor([tokens['input_ids']]))
print('Logits:', output.logits)
print('Sentiment: ', sentiment_labels[np.argmax(output.logits.tolist())])

Logits: tensor([[ 5.5690, -2.5438, -3.1783]], grad_fn=<AddmmBackward0>)
Sentiment:  negative


In [51]:
tokens = finetuned_rest_asc_tokenizer('very terrible service, price is high, but taste is good.', 'price', is_split_into_words=False)
output = finetuned_rest_asc_model(torch.tensor([tokens['input_ids']]))
print('Logits:', output.logits)
print('Sentiment: ', sentiment_labels[np.argmax(output.logits.tolist())])

Logits: tensor([[ 5.4031, -2.1838, -3.3275]], grad_fn=<AddmmBackward0>)
Sentiment:  negative


In [52]:
tokens = finetuned_rest_asc_tokenizer('very terrible service, price is high, but taste is good.', 'taste', is_split_into_words=False)
output = finetuned_rest_asc_model(torch.tensor([tokens['input_ids']]))
print('Logits:', output.logits)
print('Sentiment: ', sentiment_labels[np.argmax(output.logits.tolist())])

Logits: tensor([[-3.0090,  5.7663, -3.1077]], grad_fn=<AddmmBackward0>)
Sentiment:  positive


### Evaluation

In [28]:
rest_asc_test_file = '/Users/sean.huang/Documents/my/books and courses/ztgg/Shared Code/BERT-for-ABSA/asc/rest/test.json'
rest_asc_test_samples = json.load(open(rest_asc_test_file))
len(rest_asc_test_samples)

1120

In [57]:
def asc(sentence, term):

    tokens = finetuned_rest_asc_tokenizer(sentence, term, is_split_into_words=False)
    output = finetuned_rest_asc_model(torch.tensor([tokens['input_ids']]))
    
    return sentiment_labels[np.argmax(output.logits.tolist())]

asc('very terrible service, price is high, but taste is good.', 'taste')

'positive'

In [58]:
y_pred = []
y_true = []

for key in rest_asc_test_samples.keys():
    y_pred.append(asc(rest_asc_test_samples[key]['sentence'], rest_asc_test_samples[key]['term']))
    # print(rest_asc_test_samples[key]['sentence'])
    # print(rest_asc_test_samples[key]['term'])
    # print(asc(rest_asc_test_samples[key]['sentence'], rest_asc_test_samples[key]['term']))
    y_true.append(rest_asc_test_samples[key]['polarity'])
    

In [59]:
print(y_true[:10], y_true[:10])
print(y_pred[:10], y_pred[:10])

['positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative'] ['positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative']
['positive', 'positive', 'neutral', 'neutral', 'positive', 'negative', 'neutral', 'negative', 'negative', 'neutral'] ['positive', 'positive', 'neutral', 'neutral', 'positive', 'negative', 'neutral', 'negative', 'negative', 'neutral']


In [66]:
Counter(y_true)

Counter({'positive': 728, 'negative': 196, 'neutral': 196})

In [67]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_true, y_pred, labels=['negative', 'positive', 'neutral'])

(array([0.79787234, 0.89182058, 0.59770115]),
 array([0.76530612, 0.92857143, 0.53061224]),
 array([0.78125   , 0.90982503, 0.56216216]),
 array([196, 728, 196]))

In [65]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, y_pred, labels=['negative', 'positive', 'neutral'])

array([[150,  15,  31],
       [ 13, 676,  39],
       [ 25,  67, 104]])

In [69]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true, y_pred)

0.8303571428571429

## Home Work: do laptop ASC